### Tools

In [1]:
#%pip install -U langchain-community langgraph langchain-openai tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 8.7 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.21
    Uninstalling langchain-core-0.2.21:
      Successfully uninstalled langchain-core-0.2.21
  Attempting uninstall: langgraph
    Found existing installation: langgraph 0.1.8
    Uninstalling langgraph-0.1.8:
      Successfully uninstalled langgraph-0.1.8
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.2.7
    Uninstalling langchain-community-0.2.7:
      Successfully uninstalled langchain-community-0.2.7
Note: you may need to restart the kernel to use updated packages.


In [17]:
import getpass
import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [1]:
import getpass
import os

# os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

# define search tool

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Los Angeles")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

/Users/rickchakra/Projects/aiug_workshop/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Los Angeles', 'region': 'California', 'country': 'United States of America', 'lat': 34.05, 'lon': -118.24, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1723598355, 'localtime': '2024-08-13 18:19'}, 'current': {'last_updated_epoch': 1723598100, 'last_updated': '2024-08-13 18:15', 'temp_c': 24.4, 'temp_f': 75.9, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 12.5, 'wind_kph': 20.2, 'wind_degree': 260, 'wind_dir': 'W', 'pressure_mb': 1013.0, 'pressure_in': 29.9, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 64, 'cloud': 0, 'feelslike_c': 25.3, 'feelslike_f': 77.5, 'windchill_c': 27.6, 'windchill_f': 81.6, 'heatindex_c': 27.9, 'heatindex_f': 82.2, 'dewpoint_c': 14.9, 'dewpoint_f': 58.8, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 7.0, 'gust_mph': 17.0, 'gust_kph': 27.4}}"}, {'url': 'https://www.timeanddate.com/weather/@z-us-90023/e

### Bind model with tools

In [5]:
#pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [7]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

# initialize model
model = ChatOpenAI(model="gpt-4o")

In [8]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [9]:
# bind tools with model
model_with_tools = model.bind_tools(tools)

In [10]:
# check response of model with search tool (no tool call)
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [11]:
# check response of model with search tool (tool call)
response = model_with_tools.invoke([HumanMessage(content="What's the weather in San Francisco?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_fuPykTFGJw5N3uu9f9QFtPCW', 'type': 'tool_call'}]


### Using a prebuilt react agent with tools

In [12]:
from langgraph.prebuilt import create_react_agent

#use prebuilt ReAct agent
agent_executor = create_react_agent(model, tools)

In [13]:
# test agent response (no tool call)
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='d6b509d9-0fff-488f-ad35-68f9737be000'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27', 'finish_reason': 'stop', 'logprobs': None}, id='run-484c1aa0-12bc-4bec-a625-d6d39cc85309-0', usage_metadata={'input_tokens': 81, 'output_tokens': 10, 'total_tokens': 91})]

In [16]:
# test agent response (tool call)
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="What's the weather in San Francisco and if it's not raining, are there any event happening today (August 13 2024)?")]}
)
response["messages"]

[HumanMessage(content="What's the weather in San Francisco and if it's not raining, are there any event happening today (August 13 2024)?", id='5c7f63c3-51cc-452d-94ab-909d76bf3e55'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fiDFOmz7sxzYZmOm08pP10FO', 'function': {'arguments': '{"query": "San Francisco weather August 13 2024"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_ATSF9Y33SBJbW7jqpzOL9t4x', 'function': {'arguments': '{"query": "Events in San Francisco August 13 2024"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 105, 'total_tokens': 174}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c44d3455-2ad9-4884-96ac-04d862e2e0f0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather August 13 2024'}, 'id': 'cal